## Set up network and install software

In [ ]:
from state import *
from IPython.core.getipython import get_ipython

# Import libraries
%run ../setup/include/include_libraries.py
# Import selected slice name
%run ../slice_info/selected_slice.py


# Import topology_variables

# Use the button in the GUI automatically does it for you
if os.getenv('SELECTED_SLICE') is not None:
    SELECTED_SLICE = os.getenv('SELECTED_SLICE')

# If you manually run the notebook, please specify the slice name in the line below
# SELECTED_SLICE = 'slice_name'

path = f'../slice_info/{SELECTED_SLICE}/topology_variables.ipynb'
get_ipython().run_line_magic('run', path)

import write_value


In [ ]:
try:
    slice = fablib.get_slice(name=selected_slice)
    # install ovs and net-tools
    for node in slice.get_nodes():
        if node.get_name().startswith("bridge"):
            stdout, stderr = node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade', quiet=True) 
            stdout, stderr = node.execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common', quiet=True)
            stdout, stderr = node.execute('sudo apt-get -y install net-tools', quiet=True)
            print(f"done bridge: {node.get_name()}")
    print("Done")
    
    # Create a new bridge, enable the spanning tree protocol on necessary interfaces
    bridge1 = slice.get_node(name=node3_name)
    stdout, stderr = bridge1.execute('sudo ovs-vsctl add-br br0')
    for interface in bridge1.get_interfaces():
        stdout, stderr = bridge1.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
        #Remove IP addresses for all interfaces
        stdout, stderr = bridge1.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
    #bring the bridge up
    stdout, stderr = bridge1.execute('sudo ifconfig br0 up')
    print("Done")
    
    # enable spanning tree and confirm
    stdout, stderr = bridge1.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')
    stdout, stderr = bridge1.execute('sudo ovs-appctl stp/show')
    
    # set up host nodes
    host1 = slice.get_node(name=node1_name)
    host2 = slice.get_node(name=node2_name)
    stdout, stderr = host1.execute('sudo apt-get -y install net-tools', quiet=True)
    stdout, stderr = host2.execute('sudo apt-get -y install net-tools', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip link set dev {host1.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip link set dev {host2.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip addr add 10.10.10.1/24 dev {host1.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip addr add 10.10.10.2/24 dev {host2.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)
    
    
    success = True
except Exception as e:
    print(f"Exception: {e}")
    success = False
    raise SystemExit("Stopping notebook execution due to exception")

## Update state

In [ ]:
if success:
    update_state_local(selected_slice, 'SLICE_CONFIGURED')
    update_state_in_fim(selected_slice, read_state_from_local(selected_slice))
else:
    print ('State is not updated due to error.')